<a href="https://colab.research.google.com/github/NhuThai006/EULERIAN_CIRCUITS_TO-ASSIST_POLICE_IN_STREET_INSPECTION_IN_HCMC/blob/main/HUFFMAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import heapq
import os
import pickle
import struct
from google.colab import files

# --- NODE ---
class Node:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq


# --- HÀM ---

def build_frequency_table(data_bytes):
    """Đếm tần suất xuất hiện của các byte"""
    freq = {}
    for byte in data_bytes:
        freq[byte] = freq.get(byte, 0) + 1
    return freq

def build_huffman_tree(freq_map):
    """Xây dựng cây Huffman từ bảng tần suất"""
    heap = []
    for char, freq in freq_map.items():
        heapq.heappush(heap, Node(char, freq))

    if not heap: return None
    if len(heap) == 1: # Trường hợp file chỉ có 1 loại byte
        node = heapq.heappop(heap)
        root = Node(None, node.freq)
        root.left = node
        return root

    while len(heap) > 1:
        node1 = heapq.heappop(heap)
        node2 = heapq.heappop(heap)
        merged = Node(None, node1.freq + node2.freq)
        merged.left = node1
        merged.right = node2
        heapq.heappush(heap, merged)
    return heap[0]

def build_codes(node, current_code, codes):
    """Tạo bảng mã bit cho từng byte"""
    if node is None: return
    if node.char is not None:
        codes[node.char] = current_code
        return
    build_codes(node.left, current_code + "0", codes)
    build_codes(node.right, current_code + "1", codes)


def pad_encoded_text(encoded_text):
    """Thêm bit 0 vào cuối để đủ 8 bit (1 byte)"""
    extra_padding = 8 - len(encoded_text) % 8
    for i in range(extra_padding):
        encoded_text += "0"

    padded_info = "{0:08b}".format(extra_padding)
    return padded_info + encoded_text

def get_byte_array(padded_encoded_text):
    """Chuyển chuỗi bit '010101' thành mảng byte thực sự để ghi file"""
    if len(padded_encoded_text) % 8 != 0:
        print("Lỗi padding!")
        return bytearray()

    b = bytearray()
    for i in range(0, len(padded_encoded_text), 8):
        byte = padded_encoded_text[i:i+8]
        b.append(int(byte, 2))
    return b

def remove_padding(encoded_text):
    """Loại bỏ các bit đệm khi giải nén"""
    padded_info = encoded_text[:8]
    extra_padding = int(padded_info, 2)
    encoded_text = encoded_text[8:]
    encoded_text = encoded_text[:-extra_padding]
    return encoded_text


# --- NÉN, GIẢI NÉN FILE ---

def compress_file_binary(filepath):
    print(f"-> Đang đọc file: {filepath} (Chế độ Binary)...")
    filename, file_extension = os.path.splitext(filepath)

    with open(filepath, 'rb') as file:
        file_bytes = file.read() # Đọc file dưới dạng bytes

    # freq map, Huffman tree
    freq_map = build_frequency_table(file_bytes)
    tree = build_huffman_tree(freq_map)

    # code table
    codes = {}
    build_codes(tree, "", codes)


    encoded_text = ""
    temp_list = [codes[byte] for byte in file_bytes]
    encoded_text = "".join(temp_list)

    padded_encoded_text = pad_encoded_text(encoded_text)
    b_array = get_byte_array(padded_encoded_text)
    output_path = filename + ".huf"

    # Dùng pickle để lưu header (đuôi file + freq table) cho tiện
    header = {
        "ext": file_extension,
        "freq": freq_map
    }

    header_bytes = pickle.dumps(header)
    header_len = len(header_bytes)

    with open(output_path, 'wb') as file:
        file.write(struct.pack('I', header_len))
        file.write(header_bytes)
        file.write(bytes(b_array))

    print(f"✅ Đã nén xong: {output_path}")
    files.download(output_path)

def decompress_file_binary(filepath):
    print(f"-> Đang giải nén file: {filepath}...")

    with open(filepath, 'rb') as file:
        header_len_data = file.read(4)
        if not header_len_data: return
        header_len = struct.unpack('I', header_len_data)[0]

        header_bytes = file.read(header_len)
        header = pickle.loads(header_bytes)

        original_ext = header["ext"]
        freq_map = header["freq"]

        body_bytes = file.read()

    tree = build_huffman_tree(freq_map)

    bit_string = ""
    bit_string = "".join(f"{byte:08b}" for byte in body_bytes)
    encoded_text = remove_padding(bit_string)

    #Decode
    decoded_bytes = bytearray()
    current_node = tree

    for bit in encoded_text:
        if bit == '0':
            current_node = current_node.left
        else:
            current_node = current_node.right

        if current_node.char is not None:
            decoded_bytes.append(current_node.char)
            current_node = tree
    original_filename = os.path.splitext(filepath)[0] + "_restored" + original_ext

    with open(original_filename, 'wb') as file:
        file.write(decoded_bytes)

    print(f"✅ Đã giải nén và phục hồi định dạng gốc: {original_filename}")
    files.download(original_filename)

# --- MENU ---
def main_menu():
    while True:
        print("\n--- EXACT FORMAT HUFFMAN (BINARY MODE) ---")
        print("1. Upload file bất kỳ để NÉN (.docx, .mp4, .jpg...)")
        print("2. Upload file .huf để GIẢI NÉN (về định dạng gốc)")
        print("3. Thoát")

        c = input("Chọn: ")

        if c == '1':
            print(">>> Chọn file cần nén:")
            uploaded = files.upload()
            for fname in uploaded.keys():
                compress_file_binary(fname)

        elif c == '2':
            print(">>> Chọn file .huf cần giải nén:")
            uploaded = files.upload()
            for fname in uploaded.keys():
                if fname.endswith(".huf"):
                    decompress_file_binary(fname)
                else:
                    print("Vui lòng chọn file có đuôi .huf")

        elif c == '3':
            break
        else:
            print("Sai cú pháp!")

if __name__ == "__main__":
    main_menu()


--- EXACT FORMAT HUFFMAN (BINARY MODE) ---
1. Upload file bất kỳ để NÉN (.docx, .mp4, .jpg...)
2. Upload file .huf để GIẢI NÉN (về định dạng gốc)
3. Thoát
Chọn: 1
>>> Chọn file cần nén:


Saving test 2.docx to test 2.docx
-> Đang đọc file: test 2.docx (Chế độ Binary)...
✅ Đã nén xong: test 2.huf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


--- EXACT FORMAT HUFFMAN (BINARY MODE) ---
1. Upload file bất kỳ để NÉN (.docx, .mp4, .jpg...)
2. Upload file .huf để GIẢI NÉN (về định dạng gốc)
3. Thoát
Chọn: 3
